## Transactions

In [ ]:
%%capture
!pip install selenium

In [ ]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import json  

class TxParseVerification:

    # Locate the textarea input element and the button (using class names)
    xpath_textarea = '/html/body/div/div[2]/div[2]/main/div/div/div[1]/div[1]/textarea'
    xpath_decode = '/html/body/div/div[2]/div[2]/main/div/div/div[1]/div[1]/div/button[1]'
    xpath_parsed_tx = '/html/body/div/div[2]/div[2]/main/div/div/div[1]/div[2]/div[2]/pre'
  
    def __init__(self,raw_tx, url="https://www.blockchain.com/explorer/assets/btc/decode-transaction"):
        self.raw_tx = raw_tx
        self.url = url 
        options = Options()
        options.add_argument("--headless")  # Runs Chrome in headless mode
        options.add_argument("--no-sandbox")  # For environments like Docker
        options.add_argument("--disable-dev-shm-usage")  # To avoid certain ChromeDriver issues
        self.driver = webdriver.Firefox(options=options)

    def __call__(self,) -> dict: 
        self.driver.get(self.url)
        time.sleep(3)
        WebDriverWait(self.driver, 3).until(EC.element_to_be_clickable((By.XPATH, TxParseVerification.xpath_textarea))).send_keys(self.raw_tx)
        self.driver.find_element(By.XPATH, TxParseVerification.xpath_decode).click()
        result = json.loads(self.driver.find_element(By.XPATH, TxParseVerification.xpath_parsed_tx).text)
        self.driver.quit()
        return result 



raw_tx  = (
 '0100000001813f79011acb80925dfe69b3def355fe914bd1d9'
 '6a3f5f71bf8303c6a989c7d1000000006b483045022100ed'
 '81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf213'
 '20b0277457c98f02207a986d955c6e0cb35d446a89d3f5610'
 '0f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3'
 '624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b27'
 '8afeffffff02a135ef01000000001976a914bc3b654dca7e56b'
 '04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141'
 'c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')  # Replace with your actual raw transaction


NETWORK_CONNECTED = False 
try: 
    result = TxParseVerification(raw_tx)()
    NETWORK_CONNECTED =True 
except: 
    pass  

print(NETWORK_CONNECTED)

In [ ]:
result

In [ ]:
result['version']
result['ins'][0]['txid']
result['ins'].__len__()
result['ins'][0]['sequence']
result['outs'].__len__()
result['outs'][0]['script']['hex']
result['outs'][0]['value']
# result['outs'][0]['hex']

In [ ]:
tx_serialized = (
 '0100000001813f79011acb80925dfe69b3def355fe914bd1d9'
 '6a3f5f71bf8303c6a989c7d1000000006b483045022100ed'
 '81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf213'
 '20b0277457c98f02207a986d955c6e0cb35d446a89d3f5610'
 '0f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3'
 '624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b27'
 '8afeffffff02a135ef01000000001976a914bc3b654dca7e56b'
 '04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141'
 'c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')


from io import BytesIO
from btclib.transaction import Tx 



# script_hex = ('6b483045022100ed81ff192e75a3fd2304004dcadb746fa5e24c50'
#               '31ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3'
#               'f56100f4d7f67801c31967743a9c8e10615bed01210349fc4e631e'
#               '3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a')

# bs1 = BytesIO(bytes.fromhex(script_hex))
s = BytesIO(bytes.fromhex(tx_serialized))
tx =  Tx.parse(s)

# tx.__dict__

tx.version == result['version']
tx.tx_ins[0].prev_tx.hex() == result['ins'][0]['txid'] # d1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81
tx.tx_ins[0].sequence == result['ins'][0]['sequence'] # 4294967294
tx.tx_outs[0].amount == result['outs'][0]['value'] #  32454049
tx.tx_outs[1].amount == result['outs'][1]['value'] #  32454049
tx.tx_outs[0].script_pubkey ==  result['outs'][0]['script']['hex'] # '76a914bc3b654dca7e56b04dca18f2566cdaf02e8d9ada88ac'
tx.tx_outs[1].script_pubkey ==  result['outs'][1]['script']['hex'] # '76a9141c4bc762dd5423e332166702cb75f40df79fea1288ac'

In [3]:


tx_hex = (
 '0100000001813f79011acb80925dfe69b3def355fe914bd1d9'
 '6a3f5f71bf8303c6a989c7d1000000006b483045022100ed'
 '81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf213'
 '20b0277457c98f02207a986d955c6e0cb35d446a89d3f5610'
 '0f4d7f67801c31967743a9c8e10615bed01210349fc4e631e3'
 '624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b27'
 '8afeffffff02a135ef01000000001976a914bc3b654dca7e56b'
 '04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141'
 'c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')


from btclib.transaction import Tx
from io import  BytesIO
tx = Tx.parse(BytesIO(bytes.fromhex(tx_hex)))

tx.serialize().hex() == tx_hex


 of type <class 'btclib.script.Script'>


True

In [ ]:
expect = 'd1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81'
tx.tx_ins[0].prev_index == 0 
tx.tx_ins[0].prev_tx.hex() == expect 
tx.tx_ins.__len__() == 1 

# want = bytes.fromhex()

# want.hex()

In [ ]:

from io import BytesIO


s = BytesIO(bytes.fromhex(('a135ef01000000001976a914bc3b654dca7e56b'
 '04dca18f2566cdaf02e8d9ada88ac99c39800000000001976a9141'
 'c4bc762dd5423e332166702cb75f40df79fea1288ac19430600')))

from btclib.transaction import TxOut


TxOut.parse(s)


In [ ]:
tx.version

'd1c789a9c60383bf715f3f6ad9d14b91fe55f3deb369fe5d9280cb1a01793f81'.__len__()

In [ ]:
t# from btclib.transactions import TxFetcher, Tx,TxIn,TxOut
2**64-1

In [ ]:
import IPython
IPython.paths.get_ipython_dir()
print("Hello")

In [ ]:
int(1844674407370955000).to_bytes(8, 'little').hex()

In [ ]:


from io import BytesIO
stream_str = BytesIO
t = bytes.fromhex( 'FD10')  # 16
t = bytes.fromhex('FE2b020000') # 555
t = bytes.fromhex('FFcb10c7bab88d0600') #  1844674407370955

s = BytesIO(t) # .read(1)[0]

from btclib.transaction import Varint


Varint.decode(BytesIO(bytes.fromhex(Varint.encode(555).hex())))



In [ ]:

from io import BytesIO
stream_str = BytesIO
t = bytes.fromhex('FFcb10c7bab88d0600') #  1844674407370955

s= BytesIO(t).read(1) # .read(1)[0]
print(s)
s= BytesIO(t).read(3)
print(s)


In [ ]:
from io import BytesIO
from btclib.script import Script  # <1>

# stream = BytesIO(bytes.fromhex(script_hex))
# script_sig = Script.parse(stream)
# print(script_sig)
script_hex
#3045022100ed81ff192e75a3fd2304004dcadb746fa5e24c5031ccfcf21320b0277457c98f02207a986d955c6e0cb35d446a89d3f56100f4d7f67801c31967743a9c8e10615bed010349fc4e631e3624a545de3f89f5d8684c7b8138bd94bdd531d2e213bf016b278a